In [299]:
import os
import json
import pandas as pd
import traceback

In [300]:
from dotenv import load_dotenv

load_dotenv() 

True

In [301]:
KEY=os.getenv("HUGGING_FACE_API_KEY")

In [302]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

In [303]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="conversational", 
    provider="fireworks-ai",
    max_new_tokens=512
)
chat = ChatHuggingFace(llm=llm, verbose=True)

In [304]:
from langchain_core.messages import SystemMessage

In [305]:
system_msg = SystemMessage(content="/no_think Answer very briefly and do not explain your reasoning.")

In [306]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate

In [307]:
TEMPLATE_QUIZ_NEW = """\
{system_msg}
Context: {text}
Your task is to write exactly {number} multiple-choice questions based on the above content. The questions should be appropriate for {subject} students and written in a {difficulty} difficulty.
Return ONLY a JSON object matching the format shown in RESPONSE_JSON below. Do not include any extra explanation.
### RESPONSE_JSON
{response_json}
"""

In [308]:
quiz_prompt_new = PromptTemplate(
    input_variables=["system_msg", "text", "number", "subject", "difficulty", "response_json"],
    template=TEMPLATE_QUIZ_NEW,
)

In [309]:
quiz_chain = LLMChain(llm=chat, prompt=quiz_prompt_new, output_key="quiz")

In [310]:
TEMPLATE_REVIEW_NEW = """\
{system_msg}
Below is a quiz for {subject} students. Review its difficulty in no more than 50 words. If any question is not suitable, rewrite only the problem parts in a suitable difficulty.
Quiz:
{quiz}
"""

In [311]:
review_prompt_new = PromptTemplate(
    input_variables=["system_msg", "subject", "quiz"],
    template=TEMPLATE_REVIEW_NEW,
)

In [312]:
review_chain = LLMChain(llm=chat, prompt=review_prompt_new, output_key="review")

In [313]:
combined_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["system_msg", "text", "number", "subject", "difficulty","response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [314]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [315]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [316]:
with open("../data.txt", 'r') as file:
    TEXT = file.read()

In [317]:
input_data = {
    "system_msg": system_msg.content,
    "text": TEXT,
    "number":"2",
    "subject":"Generative AI",
    "difficulty":"intermediate",
    "response_json": json.dumps(RESPONSE_JSON)
}

In [318]:
response=combined_chain(input_data)



> Entering new SequentialChain chain...

> Finished chain.


In [321]:
print(response["quiz"].strip())

{
"1": {"mcq": "What is the primary difference between Generative AI and traditional AI systems?", "options": {"a": "Generative AI can only analyze data, while traditional AI can classify data", "b": "Generative AI can create new content, while traditional AI can only analyze data", "c": "Generative AI is faster than traditional AI, while traditional AI is more accurate", "d": "Generative AI is only used for image generation, while traditional AI is used for text classification"}, "correct": "b"},
"2": {"mcq": "Which Generative AI model is known for its ability to generate realistic images and videos?", "options": {"a": "Transformers", "b": "Diffusion Models", "c": "Variational Autoencoders (VAEs)", "d": "Generative Adversarial Networks (GANs)"}, "correct": "d"}
}


In [322]:
quiz=response.get("quiz")

In [323]:
quiz=json.loads(quiz)

In [324]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [325]:
quiz=pd.DataFrame(quiz_table_data)

In [327]:
quiz.to_csv("quiz.csv", index=False)